In [1]:
import argparse

from config import args_dict
from model import T5NerFineTuner
from dataset import T5NerFineTunerDataModule

import pytorch_lightning as pl


In [2]:
from setproctitle import setproctitle
from utils.set_seed import set_seed

setproctitle("taejung_t2t_ner_train")
set_seed(42)


In [3]:
args_dict.update({"output_dir": "checkpoints", "max_epochs": 15})
args = argparse.Namespace(**args_dict)
print(args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir,
    monitor="val_loss",
    mode="min",
    save_top_k=5,
    filename="epoch={epoch}-step={step}-val_loss={val_loss:.2f}",
)
callbacks = [checkpoint_callback]

trainer = pl.Trainer.from_argparse_args(
    args,
    callbacks=callbacks,
    auto_lr_find="learning_rate",
    auto_scale_batch_size=True,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
datamodule = T5NerFineTunerDataModule(**args_dict)

In [5]:
datamodule.prepare_data()
datamodule.setup()

캐시된 데이터셋 사용


Loading cached split indices for dataset at cached_dataset/train/cache-f4cfe387481aeba5.arrow and cached_dataset/train/cache-80cbb823f257c523.arrow


In [6]:
total_steps = len(datamodule.train_dataloader())
model = T5NerFineTuner(total_steps=total_steps, **args_dict)

"accelerator":                 gpu
"adam_epsilon":                1e-08
"batch_size":                  8
"cached_dataset_path":         cached_dataset
"devices":                     1
"gradient_accumulation_steps": 16
"kfold":                       10
"learning_rate":               0.0003
"max_grad_norm":               1.0
"max_seq_length":              256
"model_name_or_path":          /home/work/team01/model/kt-ulm-base
"num_train_epochs":            2
"output_dir":                  t5-ner
"seed":                        42
"tokenizer_name_or_path":      /home/work/team01/model/kt-ulm-base
"total_steps":                 12482
"warmup_steps":                0
"weight_decay":                0.0


In [7]:
trainer.tune(model, datamodule=datamodule)

캐시된 데이터셋 사용


Loading cached split indices for dataset at cached_dataset/train/cache-64601b7c036c31d2.arrow and cached_dataset/train/cache-89610145d16d25e2.arrow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/work/.conda/envs/htj/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/work/.conda/envs/htj/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


캐시된 데이터셋 사용


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8


캐시된 데이터셋 사용


...

캐시된 데이터셋 사용


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.002754228703338169
Restoring states from the checkpoint path at /home/work/team01/ICU_models/template-ner_T/.lr_find_3509f9bd-0c39-4ab2-8994-c77a3baadafe.ckpt


{'scale_batch_size': 16,
 'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7f1bb32db7c0>}

In [8]:
trainer.fit(model, datamodule=datamodule)

캐시된 데이터셋 사용


Loading cached split indices for dataset at cached_dataset/train/cache-64601b7c036c31d2.arrow and cached_dataset/train/cache-89610145d16d25e2.arrow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 262 M 
-----------------------------------------------------
262 M     Trainable params
0         Non-trainable params
262 M     Total params
1,049.275 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

: 

: 